# Opioids in the USA
> Source: [The Washington Post](https://www.kaggle.com/paultimothymooney/pain-pills-in-the-usa/kernels)

## Notebook Setup

In [11]:
import pandas as pd 
import dask.dataframe as dd
import os
from tqdm import tqdm

In [12]:
# the dataset that we're using is 80GB
# check out this resource
# https://www.kaggle.com/szelee/how-to-import-a-csv-file-of-55-million-rows
path = '../data/arcos_all_washpost.tsv'

## Explore Data

In [13]:
# peep at the data
temp_data = pd.read_csv(path, nrows=15, sep='\t')

In [14]:
# check out column names
for column in temp_data.columns:
	print(column)

REPORTER_DEA_NO
REPORTER_BUS_ACT
REPORTER_NAME
REPORTER_ADDL_CO_INFO
REPORTER_ADDRESS1
REPORTER_ADDRESS2
REPORTER_CITY
REPORTER_STATE
REPORTER_ZIP
REPORTER_COUNTY
BUYER_DEA_NO
BUYER_BUS_ACT
BUYER_NAME
BUYER_ADDL_CO_INFO
BUYER_ADDRESS1
BUYER_ADDRESS2
BUYER_CITY
BUYER_STATE
BUYER_ZIP
BUYER_COUNTY
TRANSACTION_CODE
DRUG_CODE
NDC_NO
DRUG_NAME
QUANTITY
UNIT
ACTION_INDICATOR
ORDER_FORM_NO
CORRECTION_NO
STRENGTH
TRANSACTION_DATE
CALC_BASE_WT_IN_GM
DOSAGE_UNIT
TRANSACTION_ID
Product_Name
Ingredient_Name
Measure
MME_Conversion_Factor
Combined_Labeler_Name
Revised_Company_Name
Reporter_family
dos_str


In [23]:
# get a feel for each column's data type
temp_data.T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
REPORTER_DEA_NO,PA0006836,PA0021179,PA0021179,PA0021179,PA0021179,PA0021179,PA0021179,PA0021179,PA0021179,PA0021179,PA0021179,PA0021179,PA0021179,PA0021179,PA0021179
REPORTER_BUS_ACT,DISTRIBUTOR,DISTRIBUTOR,DISTRIBUTOR,DISTRIBUTOR,DISTRIBUTOR,DISTRIBUTOR,DISTRIBUTOR,DISTRIBUTOR,DISTRIBUTOR,DISTRIBUTOR,DISTRIBUTOR,DISTRIBUTOR,DISTRIBUTOR,DISTRIBUTOR,DISTRIBUTOR
REPORTER_NAME,ACE SURGICAL SUPPLY CO INC,APOTHECA INC,APOTHECA INC,APOTHECA INC,APOTHECA INC,APOTHECA INC,APOTHECA INC,APOTHECA INC,APOTHECA INC,APOTHECA INC,APOTHECA INC,APOTHECA INC,APOTHECA INC,APOTHECA INC,APOTHECA INC
REPORTER_ADDL_CO_INFO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
REPORTER_ADDRESS1,1034 PEARL STREET,1622 N 16TH ST,1622 N 16TH ST,1622 N 16TH ST,1622 N 16TH ST,1622 N 16TH ST,1622 N 16TH ST,1622 N 16TH ST,1622 N 16TH ST,1622 N 16TH ST,1622 N 16TH ST,1622 N 16TH ST,1622 N 16TH ST,1622 N 16TH ST,1622 N 16TH ST
REPORTER_ADDRESS2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
REPORTER_CITY,BROCKTON,PHOENIX,PHOENIX,PHOENIX,PHOENIX,PHOENIX,PHOENIX,PHOENIX,PHOENIX,PHOENIX,PHOENIX,PHOENIX,PHOENIX,PHOENIX,PHOENIX
REPORTER_STATE,MA,AZ,AZ,AZ,AZ,AZ,AZ,AZ,AZ,AZ,AZ,AZ,AZ,AZ,AZ
REPORTER_ZIP,2301,85006,85006,85006,85006,85006,85006,85006,85006,85006,85006,85006,85006,85006,85006
REPORTER_COUNTY,PLYMOUTH,MARICOPA,MARICOPA,MARICOPA,MARICOPA,MARICOPA,MARICOPA,MARICOPA,MARICOPA,MARICOPA,MARICOPA,MARICOPA,MARICOPA,MARICOPA,MARICOPA


In [16]:
# some of this data will get pretty big;
# it will be important to practice dropping unused data.
del temp_data

### Outlining Data Columns
There's many columns here.
It will be important to outline what each of them actually represent.
Some of these are actually self-explanatory, but others might make do with some greater understanding.

#### Important Terminology
- **DEA**: Drug Enforcement Administration
- **NDC**: National Drug Code
- **BUS_ACT**: Seems to stand for business activity.
- **NO**: Number
- **ADDL_CO_INFO**: Additional Contact Information
- **MME**: Morphine Milligram Equivalent

#### Column Descriptions
- REPORTER_DEA_NO
	- appears to be a unique identifier for a buyer or reporter. 
- REPORTER_BUS_ACT
	- appears to be a class of reporter.
		options include:
		- DISTRIBUTOR
		- MANUFACTURER
		- REVERSE DISTRIB
- ~~REPORTER_NAME~~
- ~~REPORTER_ADDL_CO_INFO~~
- ~~REPORTER_ADDRESS1~~
- ~~REPORTER_ADDRESS2~~
- ~~REPORTER_CITY~~
- ~~REPORTER_STATE~~
- ~~REPORTER_ZIP~~
- ~~REPORTER_COUNTY~~
- BUYER_DEA_NO
	- appears to be a unique identifier for a buyer or reporter.
- BUYER_BUS_ACT
	- appears to be a class of buyer.
		options include:
		- CHAIN PHARMACY
		- RETAIL PHARMACY
		- PRACTITIONER
- ~~BUYER_NAME~~
- ~~BUYER_ADDL_CO_INFO~~
- ~~BUYER_ADDRESS1~~
- ~~BUYER_ADDRESS2~~
- ~~BUYER_CITY~~
- ~~BUYER_STATE~~
- ~~BUYER_ZIP~~
- ~~BUYER_COUNTY~~
- TRANSACTION_CODE
	- Inventory Transaction Codes
		- 1: Schedule Change Inventory
		- 3: Year-End Inventory
		- 4: Year-End In-Process Inventory (manufacturers only)
		- 5: Special Inventory
		- 8: No Year-End Inventory
	- Acquisition Transaction Codes (Increases to Inventory)
		- P: Purchase or Receipt
		- R: Return
		- V: Unsolicited Return
		- G: Government Supplied
		- W: Recovered Waste (manufacturers only)
		- M: Manufactured (manufacturers only)
		- L: Reversing (manufacturers only)
		- J: Return of Sample to Inventory (manufacturers only)
	- Disposition Transaction Codes (Decreases to Inventory)
		- S: Sale, Disposition, or Transfer
		- Y: Destroyed
		- T: Theft
		- Z: Receipt by Government (seizures, samples, etc.)
		- N: Nonrecoverable Waste (manufacturers only)
		- U: Used in Production (manufacturers only)
		- Q: Sampling (manufacturers only)
		- K: Used in Preparations (manufacturers only)
	- Miscellaneous Transaction Codes
		- 7: No ARCOS Activity for the Current Reporting Period
		- F: Reorder DEA Form 333
		- X: Lost-in-Transit
- DRUG_CODE
	- this entry seems to line up with the drug name.
		it might function as some sort of low-level identifier, much like a upc.
- NDC_NO
	- this entry seems to line up with the product name.
		it might function as some sort of low-level identifier, much like a upc.
- DRUG_NAME
	- this entry lines up with ingredient name.
- QUANTITY
	- most likely, this is the number of units purchased.
- UNIT
	- `WARNING` unfortunately, this appears to be a junk column.
- ACTION_INDICATOR
	- D: delete a transaction record
	- A: adjust (revise) data in a transaction record
	- I: insert (add) a late transaction record
- ORDER_FORM_NO
	- most likely, this is some sort of identifier for a class of form.
		note that only OXYCODONE drugs require/have a form entry.
		HYDROCODONE does not use these forms.
- CORRECTION_NO
	- `WARNING` unfortunately, this appears to be a junk column.
- STRENGTH
	- `WARNING` unfortunately, this appears to be a junk column.
- ~~TRANSACTION_DATE~~
- CALC_BASE_WT_IN_GM
	- this seems to be a calculated column.
		if we can determine what the formula was for this, we can delete this entry.
- DOSAGE_UNIT
	- seems to be the number of mg of drugs that the tablet contains.
- TRANSACTION_ID
- Product_Name
	- the shelf name for the product.
- Ingredient_Name
	- the long-form chemical name for the product.
- Measure
	- because all entries are "tab", it seems like this refers to the drug being a "tablet".
- MME_Conversion_Factor
	- Morphine Milligram Equivalent
- Combined_Labeler_Name
	- seems like some sort of label company for the product itself.
- Revised_Company_Name
	- seems like some sort of parent company for the product itself.
- Reporter_family
	- seems like the umbrella company that reported this entry.
- dos_str
	- number of milligrams of the drug in a capsule

## Prep Data

## Clean Data

In [17]:
# create a dictionary of current column names,
# and match them with new, simplified titles
columns_to_rename = {
	'REPORTER_DEA_NO':       'reporter id',
	'REPORTER_BUS_ACT':      'reporter business activity',
	'REPORTER_NAME':         'reporter name',
	'REPORTER_ADDL_CO_INFO': 'reporter contact information',
	'REPORTER_ADDRESS1':     'reporter address 1',
	'REPORTER_ADDRESS2':     'reporter address 2',
	'REPORTER_CITY':         'reporter city',
	'REPORTER_STATE':        'reporter state',
	'REPORTER_ZIP':          'reporter zip',
	'REPORTER_COUNTY':       'reporter county',
	'BUYER_DEA_NO':          'buyer id',
	'BUYER_BUS_ACT':         'buyer business activity',
	'BUYER_NAME':            'buyer name',
	'BUYER_ADDL_CO_INFO':    'buyer contact information',
	'BUYER_ADDRESS1':        'buyer address 1',
	'BUYER_ADDRESS2':        'buyer address 2',
	'BUYER_CITY':            'buyer city',
	'BUYER_STATE':           'buyer state',
	'BUYER_ZIP':             'buyer zip',
	'BUYER_COUNTY':          'buyer county',
	'TRANSACTION_CODE':      'transaction code',
	'DRUG_CODE':             'drug code',
	'NDC_NO':                'national drug code',
	'DRUG_NAME':             'drug name',
	'QUANTITY':              'quantity',
	'UNIT':                  'unit',
	'ACTION_INDICATOR':      'action item',
	'ORDER_FORM_NO':         'form number',
	'CORRECTION_NO':         'correction number',
	'STRENGTH':              'junk strength',
	'TRANSACTION_DATE':      'transaction date',
	'CALC_BASE_WT_IN_GM':    'weight (grams)',
	'DOSAGE_UNIT':           'order size',
	'TRANSACTION_ID':        'transaction id',
	'Product_Name':          'product name',
	'Ingredient_Name':       'ingredient name',
	'Measure':               'drug form',
	'MME_Conversion_Factor': 'morphine milligram equivalent',
	'Combined_Labeler_Name': 'labeler name',
	'Revised_Company_Name':  'company name',
	'Reporter_family':       'reporter family',
	'dos_str':               'strength (mg)'
}

In [18]:
# create a set of current column names,
# these are columns that we want to delete
columns_to_delete = {
	# 'reporter id',
	# 'reporter business activity',
	# 'reporter name',
	# 'reporter contact information',
	# 'reporter address 1',
	# 'reporter address 2',
	# 'reporter city',
	# 'reporter state',
	# 'reporter zip',
	# 'reporter county',
	# 'buyer id',
	# 'buyer business activity',
	# 'buyer name',
	# 'buyer contact information',
	# 'buyer address 1',
	# 'buyer address 2',
	# 'buyer city',
	# 'buyer state',
	# 'buyer zip',
	# 'buyer county',
	# 'transaction code',
	# 'drug code',
	# 'national drug code',
	# 'drug name',
	# 'quantity',
	# 'unit',
	# 'action item',
	# 'form number',
	# 'correction number',
	# 'junk strength',
	# 'transaction date',
	# 'weight (grams)',
	# 'order size',
	# 'transaction id',
	# 'product name',
	# 'ingredient name',
	# 'drug form',
	# 'morphine milligram equivalent',
	# 'labeler name',
	# 'company name',
	# 'reporter family',
	# 'strength (mg)'
}

In [19]:
def rename_column(dataframe, old_name, new_name):
	dataframe[new_name] = dataframe[old_name]
	dataframe = dataframe.drop([old_name], 1)
	return dataframe

def rename_columns_by_dictionary(dataframe, dictionary):
	for key in dictionary:
		old_name = key
		new_name = dictionary[key]
		dataframe = rename_column(dataframe, old_name, new_name)
	return dataframe

data = rename_columns_by_dictionary(data, columns_to_rename)

In [22]:
def delete_column(dataframe, del_name):
	dataframe = dataframe.drop([del_name], 1)
	return dataframe

def delete_columns_by_array(dataframe, array):
	for item in array:
		del_name = item
		dataframe = delete_column(dataframe, del_name)
	return dataframe

data = delete_columns_by_array(data, columns_to_delete)